**Dataset Variables for your reference**

The dataset includes the following variables

* Date: The date of the observation.
* Recession: A binary variable indicating recession perion; 1 means it was recession, 0 means it was normal.
* Automobile_Sales: The number of vehicles sold during the period.
* GDP: The per capita GDP value in USD.
* Unemployment_Rate: The monthly unemployment rate.
* Consumer_Confidence: A synthetic index representing consumer confidence, which can impact consumer spending and automobile purchases.
* Seasonality_Weight: The weight representing the seasonality effect on automobile sales during the period.
* Price: The average vehicle price during the period.
* Advertising_Expenditure: The advertising expenditure of the company.
* Vehicle_Type: The type of vehicles sold; Supperminicar, Smallfamiliycar, Mediumfamilycar, Executivecar, Sports.
* Competition: The measure of competition in the market, such as the number of competitors or market share of major manufacturers.
* Month: Month of the observation extracted from Date.
* Year: Year of the observation extracted from Date.

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]
#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'left', 'color': "#000000", 'font-size': '24px'}),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown_statistics',
            options= [
                    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                    ],
            placeholder='Select a report type'
        )
    ]),
    html.Div([html.Label("Select Year:"), dcc.Dropdown(
            id='select_year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select a year',
        )]),
    html.Div([#TASK 2.3: Add a division for output display
    html.Div(id='output-container', className='chart-gird', style = {'display':'flex'})])
])

#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select_year', component_property='disabled'),
    Input(component_id='dropdown_statistics',component_property='value'))

def update_input_container(dropdown_statistics):
    if dropdown_statistics =='Yearly Statistics': 
        return False
    else: 
        return True

# Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select_year', component_property='value'), Input(component_id='dropdown_statistics', component_property='value')]
)
def update_output_container(select_year, dropdown_statistics):
    if dropdown_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # Create graphs for Recession Period Statistics
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Average Automobile Sales fluctuation over Recession Period"))

        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.pie(average_sales, values='Automobile_Sales', names='Vehicle_Type', title='Average Number of Vehicles Sold by Vehicle Type'))

        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].mean().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type', title='Total Expenditure Share by Vehicle Type during Recessions'))

        unemploy = recession_data.groupby(['Vehicle_Type', 'unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemploy, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', title='Effect of Unemployment Rate on Vehicle Type & Sales'))

        return [
            html.Div(className='chart-item', children=[html.Div(R_chart1,className='chart'), html.Div(R_chart2,className='chart')]),
            html.Div(className='chart-item', children=[html.Div(R_chart3,className='chart'), html.Div(R_chart4,className='chart')])
        ]

    elif dropdown_statistics == 'Yearly Statistics':
        if select_year:
            # Filter the data for the selected year
            yearly_data = data[data['Year'] == select_year]

            yas = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
            Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title='Yearly Automobile Sales',markers=True))

            mas = yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index()
            Y_chart2 = dcc.Graph(figure=px.line(mas, x='Month', y='Automobile_Sales', title='Monthly Automobile Sales'))

            avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales', title='Average Vehicles Sold by Vehicle Type'))

            exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].mean().reset_index()
            Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type', title='Total Advertisement Expenditure for each vehicle'))

            return [
                html.Div(className='chart-item', children=[html.Div(Y_chart1), html.Div(Y_chart2)]),
                html.Div(className='chart-item', children=[html.Div(Y_chart3), html.Div(Y_chart4)])
            ]

    return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
